# Tutorial 02.1 W state

In [1]:
import torch

# define the number of qubits
N = 5

# define a rank-N tensor
A = torch.zeros((2,) * N)

# assign coefficients
list = [[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]
A[list] = 1 / torch.sqrt(torch.tensor(N, dtype=torch.float))

# check whether it works
print(torch.count_nonzero(A))  # number of nonzero elements
print(A[1, 0, 0, 0, 0] * torch.sqrt(torch.tensor(N)))  # 2nd element
print(A[0, 1, 0, 0, 0] * torch.sqrt(torch.tensor(N)))  # 3rd element
print(A[0, 0, 1, 0, 0] * torch.sqrt(torch.tensor(N)))  # 4th element
print(A[0, 0, 0, 1, 0] * torch.sqrt(torch.tensor(N)))  # 5th element
print(A[0, 0, 0, 0, 1] * torch.sqrt(torch.tensor(N)))  # 6th element

tensor(5)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)


# Tutorial 02.2 Tensor contractions

In [2]:
import torch

# Define tensors
A = torch.rand(4, 5, 3)
B = torch.rand(6, 4, 5)

# Contract the first and second legs of A to the second and third legs of B, respectively.
C = torch.tensordot(A, B, dims=([0, 1], [1, 2]))
print(C)

tensor([[5.2450, 4.8354, 3.6112, 3.4941, 3.6081, 4.7894],
        [4.2956, 3.9323, 3.7275, 3.6172, 4.6672, 3.5172],
        [5.9325, 6.3375, 5.5782, 4.9800, 6.5101, 5.6090]])


In [3]:
# The contract function also provides the permutation after the contraction.
C = torch.tensordot(A, B, dims=([0, 1], [1, 2])).permute(1, 0)
print(C)

tensor([[5.2450, 4.2956, 5.9325],
        [4.8354, 3.9323, 6.3375],
        [3.6112, 3.7275, 5.5782],
        [3.4941, 3.6172, 4.9800],
        [3.6081, 4.6672, 6.5101],
        [4.7894, 3.5172, 5.6090]])


In [4]:
# If one makes any mistake, the routine gives relevant error message.
# The ranks are specified wrongly:
try:
    print(torch.tensordot(A, B, dims=([0, 1], [1, 2])))
except RuntimeError as e:
    print(e)

tensor([[5.2450, 4.8354, 3.6112, 3.4941, 3.6081, 4.7894],
        [4.2956, 3.9323, 3.7275, 3.6172, 4.6672, 3.5172],
        [5.9325, 6.3375, 5.5782, 4.9800, 6.5101, 5.6090]])


In [5]:
# The number of legs to be contracted do not match:
try:
    print(torch.tensordot(A, B, dims=([0, 1], [1])))
except RuntimeError as e:
    print(e)

both dimension lists should have same length


In [6]:
# The sizes of the legs to be contracted do not match:
try:
    torch.tensordot(A, B, dims=([0, 1], [0, 1]))
except RuntimeError as e:
    print(e)

contracted dimensions need to match, but first has size 4 in dim 0 and second has size 6 in dim 0


In [7]:
F = torch.rand(3, 4, 1, 1, 1)  # define F as rank-5

# But when we query the size of F, it shows as rank-2.
print(F.size())
print(F.ndim == 2)

# On the other hand, the singleton dimensions in the middle are not truncated:
F = torch.rand(3, 1, 4)
print(F.size())
print(F.ndim == 2)

torch.Size([3, 4, 1, 1, 1])
False
torch.Size([3, 1, 4])
False


In [8]:
import torch

d_a = 101  # d_alpha
d_b = 102  # d_beta
d_c = 103  # d_gamma
d_d = 104  # d_delta
d_m = 105  # d_mu

# tensor A(gamma, delta)
A = torch.rand(d_c, d_d)
# tensor B(alpha, mu, gamma)
B = torch.rand(d_a, d_m, d_c)
# tensor C(beta, mu, delta)
C = torch.rand(d_b, d_m, d_d)

# Way 1: contract B and C first, then contract with A (as done in the demonstration)
BC = torch.tensordot(B, C, dims=([1], [1]))  # BC(alpha, gamma, beta, delta)
ABC1 = torch.tensordot(BC, A, dims=([1, 3], [0, 1]))  # ABC(alpha, beta)

# Way 2: contract A and C first, then contract with B (as asked in the Exercise)
AC = torch.tensordot(A, C, dims=([1], [2]))  # AC(gamma, beta, mu)
ABC2 = torch.tensordot(B, AC, dims=([1, 2], [2, 0]))  # ABC(alpha, beta)

# We should always check whether two different ways give the same result.
print(torch.sum(torch.abs(ABC1 - ABC2)))  # absolute error
print(torch.mean(torch.abs((ABC1 - ABC2) / ABC1)))  # relative error

tensor(166.5938)
tensor(1.1495e-07)
